In [1]:
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn import tree
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn import metrics
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import SGDClassifier
from sklearn import svm

In [2]:
df = pd.read_csv("../../data/sea/dataset.csv")
df_1 = df.drop('title', axis=1)
df_1 = df_1.drop('abstract', axis=1)
df = df_1.drop('claim', axis=1)

In [3]:
x = df.loc[:, df.columns!='valid_patent'].astype(str)
y = df['valid_patent'].astype(int)
print(x.shape, y.shape)

(1905, 5566) (1905,)


In [4]:
x = np.array(x)
y = np.array(y)
def count_values(x):
    unique, counts = np.unique(x, return_counts=True)
    return dict(zip(unique, counts))

In [6]:
# for i in range(1,100):

total_accuracy = 0
total_precision = 0
total_recall = 0
total_f1 = 0

# splits = i
splits = 10

skf = StratifiedShuffleSplit(n_splits=splits, random_state=42, test_size=0.3)

for train_index, test_index in skf.split(x, y):
#     print("TRAIN:", train_index[:15], "\nTEST:", test_index[:15])    
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
#     print("x_train: ", count_values(x_train), "x_test: ", count_values(x_test))
#         print("y_train: ", count_values(y_train), ", y_test: ", count_values(y_test))

#         print('\nOriginal dataset shape {}'.format(Counter(y_train)))
    sm = SMOTE(random_state=42)
    x_res , y_res = sm.fit_sample(x_train, y_train)
#         print('Resampled dataset shape {}'.format(Counter(y_res)))

    from sklearn.svm import LinearSVC
    from sklearn.calibration import CalibratedClassifierCV

    svc = CalibratedClassifierCV(base_estimator=LinearSVC(penalty='l2', dual=False), cv=5)
    svc.fit(x_res, y_res)


    y_pred_tr = svc.predict(x_test)

    accuracy = metrics.accuracy_score(y_test, y_pred_tr)
    total_accuracy += float(accuracy)

    precision = metrics.precision_score(y_test, y_pred_tr)
    total_precision += float(precision)

    recall = metrics.recall_score(y_test, y_pred_tr)
    total_recall += float(recall)

    f1_score = metrics.f1_score(y_test, y_pred_tr)
    total_f1 += float(f1_score)

#     if i % 10 == 0:
    print('\nAccuracy: %.2f' % accuracy)
    print('Precision: %.2f' % precision)
    print('Recall: %.2f' % recall)
    print('F1-score: %.2f' % f1_score)
#     print('Accuracy: %.2f' % metrics.accuracy_score(y_test, y_pred_tr))
    print('===============================\n')

# print('\n**'+str(i)+' splits Averge Score**')
print('Accuracy: %.2f' % (total_accuracy/splits))
print('Precision: %.2f' % (total_precision/splits))
print('Recall: %.2f' % (total_recall/splits))
print('F1-score: %.2f' % (total_f1/splits))
print('*************************')


Accuracy: 0.75
Precision: 0.02
Recall: 0.33
F1-score: 0.04


Accuracy: 0.72
Precision: 0.02
Recall: 0.44
F1-score: 0.05


Accuracy: 0.85
Precision: 0.04
Recall: 0.33
F1-score: 0.06


Accuracy: 0.77
Precision: 0.02
Recall: 0.22
F1-score: 0.03


Accuracy: 0.71
Precision: 0.02
Recall: 0.44
F1-score: 0.05


Accuracy: 0.85
Precision: 0.01
Recall: 0.11
F1-score: 0.02


Accuracy: 0.88
Precision: 0.06
Recall: 0.44
F1-score: 0.10


Accuracy: 0.88
Precision: 0.03
Recall: 0.22
F1-score: 0.06


Accuracy: 0.71
Precision: 0.04
Recall: 0.67
F1-score: 0.07


Accuracy: 0.87
Precision: 0.04
Recall: 0.33
F1-score: 0.08

Accuracy: 0.80
Precision: 0.03
Recall: 0.36
F1-score: 0.06
*************************
